In [8]:

import pandas as pd

!pip install numerapi

from numerapi import NumerAPI

from pathlib import Path

#from sklearn.linear_model import Ridge
from sklearn.feature_selection import SelectKBest, f_regression

import pyarrow.parquet as pq
import s3fs


TOURNAMENT_NAME= "first_large"
ERA_COL = "era"
TARGET_COL = "target_nomi_20"
DATA_TYPE_COL = "data_type"
EXAMPLE_PREDS_COL = "example_preds"
PREDICTION_NAME = "prediction"

id = "OML65REYFDPC5O7N22XCRP44BG2M74XH"
key = "YSTL455VERL7WZ4D7OQ6XEYEQN2MRCCICBMILNFP3DUZC4MSAS2WSH2MV7ED6WB3"

napi = NumerAPI(public_id=id,secret_key=key)


In [9]:
from utils import save_stuff, load_stuff
# utilities


def score(df):
    return df[[TARGET_COL, PREDICTION_NAME]].corr(method="spearman")[TARGET_COL][PREDICTION_NAME]



In [12]:
current_round = napi.get_current_round(tournament=8)  # tournament 8 is the primary Numerai Tournament

print('Downloading dataset files...')



AWSAccessKeyId = os.getenv('AWSAccessKeyId')
AWSSecretKey = os.getenv('AWSSecretKey')

bucket = 'data-newyork-numerai-plus'

fs = s3fs.S3FileSystem(key=AWSAccessKeyId,secret=AWSSecretKey)

validation_data = pq.read_table('s3://' + bucket + '/' + 'numerai_validation_data.parquet', filesystem=fs).to_pandas()
tournament_data = pq.read_table('s3://' + bucket + '/' + 'numerai_tournament_data.parquet', filesystem=fs).to_pandas()

#napi.download_dataset("numerai_tournament_data.csv", f"tournament_data_{current_round}.csv")
#napi.download_dataset("numerai_validation_data.csv", f"validation_data.csv")


napi.download_dataset("example_validation_predictions.csv", "example_validation_predictions.csv")

model_name = f"model_target"
print(f"Checking for existing model '{model_name}'")
model = load_stuff(model_name)
selected_features = load_stuff('features')
if not model:
    print(f"model not found")

model_expected_features = selected_features

read_columns = model_expected_features + [ERA_COL, DATA_TYPE_COL, TARGET_COL]

#validation_data = pd.read_csv('validation_data.csv', usecols=read_columns)
validation_data[PREDICTION_NAME] = model.predict(validation_data[model_expected_features])

validation_preds = pd.read_csv('example_validation_predictions.csv')
validation_data[EXAMPLE_PREDS_COL] = validation_preds["prediction"]

validation_correlations = validation_data.groupby("era").apply(score)
print(f"On validation the correlation has mean {validation_correlations.mean()} and std {validation_correlations.std()}")


#tournament_data = pd.read_csv(f'tournament_data_{current_round}.csv')

print('Predicting on tournament data')
tournament_data[PREDICTION_NAME] = model.predict(tournament_data[model_expected_features])

tournament_data[PREDICTION_NAME].to_csv("predictions.csv")





2021-12-07 05:59:25,178 INFO numerapi.utils: target file already exists
2021-12-07 05:59:25,179 INFO numerapi.utils: download complete
2021-12-07 05:59:25,708 INFO numerapi.utils: target file already exists
2021-12-07 05:59:25,708 INFO numerapi.utils: download complete
2021-12-07 05:59:26,216 INFO numerapi.utils: target file already exists
2021-12-07 05:59:26,217 INFO numerapi.utils: download complete


Checking for existing model 'model_target'
On validation the correlation has mean 0.015585386954977862 and std 0.033450374295337995


/srv/conda/envs/saturn/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Predicting on tournament data


In [13]:

#print('uploading')

path =  "predictions.csv"

print('uploading')
submission_string = napi.upload_predictions(file_path=path,version=2)
print(submission_string)






2021-12-07 06:02:44,797 INFO numerapi.base_api: uploading predictions...


uploading


2021-12-07 06:02:46,770 ERROR numerapi.base_api: Can't update submission after deadline if previous submission was on time


ValueError: Can't update submission after deadline if previous submission was on time